### Fine Tuning a pre-trained model

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
# Importing the required libraries
from keras.models import Sequential
from scipy.misc import imread
get_ipython().magic('matplotlib inline')
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras.layers import Dense
import pandas as pd

from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np
from keras.applications.vgg16 import decode_predictions
from keras.utils.np_utils import to_categorical

from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation

from sklearn.metrics import log_loss
from keras.models import Model
from keras.utils import multi_gpu_model
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img 
from keras.preprocessing.image import ImageDataGenerator

/home/arpit/anaconda3/envs/tensorflow-gpu/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
num_classes = 10

In [4]:
model = VGG16(weights='imagenet', include_top=True)

In [5]:
model.layers.pop()

In [6]:
x = model.layers[-1].output

In [7]:
x = Dense(num_classes, activation='softmax')(x)

In [8]:
model = Model(input = model.input, output = x)

/home/arpit/anaconda3/envs/tensorflow-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [10]:
# To retain all the layers weights and retrain only top layer\

# for layer in model.layers[:-1]:
#     layer.trainable = False

In [11]:
# for layer in model.layers[:15]:
#     layer.trainable = False
    
for layer in model.layers[:-1]:
    layer.trainable = False

In [12]:

# Check the trainable status of the individual layers# Check  
for layer in model.layers:
    print(layer, layer.trainable)

<keras.engine.topology.InputLayer object at 0x7f0297f284a8> False
<keras.layers.convolutional.Conv2D object at 0x7f0297f288d0> False
<keras.layers.convolutional.Conv2D object at 0x7f0297f286d8> False
<keras.layers.pooling.MaxPooling2D object at 0x7f0297f28eb8> False
<keras.layers.convolutional.Conv2D object at 0x7f02818c80f0> False
<keras.layers.convolutional.Conv2D object at 0x7f02818c8cf8> False
<keras.layers.pooling.MaxPooling2D object at 0x7f0281872be0> False
<keras.layers.convolutional.Conv2D object at 0x7f02818819b0> False
<keras.layers.convolutional.Conv2D object at 0x7f0281898eb8> False
<keras.layers.convolutional.Conv2D object at 0x7f02818acef0> False
<keras.layers.pooling.MaxPooling2D object at 0x7f0281841f28> False
<keras.layers.convolutional.Conv2D object at 0x7f028186ca20> False
<keras.layers.convolutional.Conv2D object at 0x7f0281801390> False
<keras.layers.convolutional.Conv2D object at 0x7f0281814a90> False
<keras.layers.pooling.MaxPooling2D object at 0x7f02817c2cf8> Fa

In [13]:
# Learning rate is changed to 0.001
sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [15]:
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'data/train',  # this is the target directory
        target_size=(224, 224),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'data/validation',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical')

Found 44000 images belonging to 10 classes.
Found 5000 images belonging to 10 classes.


In [16]:
model.fit_generator(
        train_generator,
        steps_per_epoch=44000 // batch_size,
        epochs=5,
        validation_data=validation_generator,
        validation_steps=5000 // batch_size)

# model.save_weights('first_try.h5') 
# always save your weights after training or during training

Epoch 1/5
2750/2750 [==============================] - 331s 120ms/step - loss: 1.5167 - acc: 0.4952 - val_loss: 1.1815 - val_acc: 0.6733
Epoch 2/5
2750/2750 [==============================] - 327s 119ms/step - loss: 1.0400 - acc: 0.6786 - val_loss: 0.9418 - val_acc: 0.7628
Epoch 3/5
2750/2750 [==============================] - 327s 119ms/step - loss: 0.8733 - acc: 0.7394 - val_loss: 0.9142 - val_acc: 0.7125
Epoch 4/5
2750/2750 [==============================] - 328s 119ms/step - loss: 0.7770 - acc: 0.7670 - val_loss: 0.7713 - val_acc: 0.7728
Epoch 5/5
2750/2750 [==============================] - 328s 119ms/step - loss: 0.7130 - acc: 0.7874 - val_loss: 0.6844 - val_acc: 0.8177


In [17]:
test_generator = test_datagen.flow_from_directory(
    directory="data/test",
    target_size=(224, 224),
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=42
)

Found 21000 images belonging to 1 classes.


In [18]:
model.evaluate_generator(generator=validation_generator)

[0.6837912809848785, 0.818]

In [19]:
test_generator.reset()
pred=model.predict_generator(test_generator,verbose=1)

21000/21000 [==============================] - 225s 11ms/step


In [20]:
predicted_class_indices=np.argmax(pred,axis=1)

In [21]:
predicted_class_indices[:5]

array([4, 0, 9, 7, 9])

In [22]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())

In [23]:
predictions = [labels[k] for k in predicted_class_indices]

In [24]:
predictions[:5]

['4', '0', '9', '7', '9']

In [25]:
filenames=test_generator.filenames

In [26]:
results=pd.DataFrame({"filename":filenames,
                      "label":predictions})
results.to_csv("results/results_fine_tune.csv",index=False)